In [1]:
import numpy as np
import pandas as pd
import time
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import random
from pandas import get_dummies
import tensorflow as tf
%matplotlib inline


In [2]:
df = pd.read_csv("WineData/wine.txt", header=None) 
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [10]:
train = pd.read_csv("WineData/train_wine.csv", header=None) 
test = pd.read_csv("WineData/test_wine.csv", header=None) 

In [11]:
df = train.append(test)
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
23,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
24,2,12.42,2.55,2.27,22.0,90,1.68,1.84,0.66,1.42,2.70,0.86,3.30,315
25,2,12.64,1.36,2.02,16.8,100,2.02,1.41,0.53,0.62,5.75,0.98,1.59,450
26,2,11.76,2.68,2.92,20.0,103,1.75,2.03,0.60,1.05,3.80,1.23,2.50,607
27,2,11.79,2.13,2.78,28.5,92,2.13,2.24,0.58,1.76,3.00,0.97,2.44,466


In [12]:
df_norm = df[[1,2,3,4,5,6,7,8,9,10,11,12,13]].apply(lambda x: (x - x.mean()) / (x.std()))
df_norm.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.500494,-0.558616,0.230721,-1.158640,1.884033,0.803011,1.025974,-0.655083,1.212968,0.250266,0.360022,1.820315,1.004475
1,-1.666156,-0.594491,0.925132,1.952458,-0.826691,-0.604470,-0.425443,0.310681,-0.444580,-1.064514,1.763456,0.826911,-0.592754
2,-1.960728,-1.428581,0.486557,0.456738,-0.826691,0.291200,-0.025052,0.471642,-0.270101,-0.856917,0.623166,-0.432333,-0.999206
3,0.948171,0.383099,-0.244401,0.755882,-0.687680,-1.516134,-1.356352,0.391161,-0.985464,1.954290,-1.131127,-1.313803,-0.424458
4,0.359027,-0.549647,-0.829168,-0.739839,-0.409657,0.163247,0.155123,-0.735563,-0.427132,-0.480648,0.272307,0.211281,1.703064


In [13]:
target = df[[0]]
df = pd.concat([df_norm, target], axis=1)
df.columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'label']
label_cols = pd.get_dummies(df.label)
df = pd.concat([df_norm, label_cols], axis=1)
df.columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'label1', 'label2', 'label3']
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,label1,label2,label3
0,1.500494,-0.558616,0.230721,-1.158640,1.884033,0.803011,1.025974,-0.655083,1.212968,0.250266,0.360022,1.820315,1.004475,1,0,0
1,-1.666156,-0.594491,0.925132,1.952458,-0.826691,-0.604470,-0.425443,0.310681,-0.444580,-1.064514,1.763456,0.826911,-0.592754,0,1,0
2,-1.960728,-1.428581,0.486557,0.456738,-0.826691,0.291200,-0.025052,0.471642,-0.270101,-0.856917,0.623166,-0.432333,-0.999206,0,1,0
3,0.948171,0.383099,-0.244401,0.755882,-0.687680,-1.516134,-1.356352,0.391161,-0.985464,1.954290,-1.131127,-1.313803,-0.424458,0,0,1
4,0.359027,-0.549647,-0.829168,-0.739839,-0.409657,0.163247,0.155123,-0.735563,-0.427132,-0.480648,0.272307,0.211281,1.703064,1,0,0


In [14]:
train = df.iloc[:151, :]
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,label1,label2,label3
0,1.500494,-0.558616,0.230721,-1.158640,1.884033,0.803011,1.025974,-0.655083,1.212968,0.250266,0.360022,1.820315,1.004475,1,0,0
1,-1.666156,-0.594491,0.925132,1.952458,-0.826691,-0.604470,-0.425443,0.310681,-0.444580,-1.064514,1.763456,0.826911,-0.592754,0,1,0
2,-1.960728,-1.428581,0.486557,0.456738,-0.826691,0.291200,-0.025052,0.471642,-0.270101,-0.856917,0.623166,-0.432333,-0.999206,0,1,0
3,0.948171,0.383099,-0.244401,0.755882,-0.687680,-1.516134,-1.356352,0.391161,-0.985464,1.954290,-1.131127,-1.313803,-0.424458,0,0,1
4,0.359027,-0.549647,-0.829168,-0.739839,-0.409657,0.163247,0.155123,-0.735563,-0.427132,-0.480648,0.272307,0.211281,1.703064,1,0,0


In [15]:
test = df.iloc[151: , :]
test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,label1,label2,label3
0,1.500494,-0.558616,0.230721,-1.158640,1.884033,0.803011,1.025974,-0.655083,1.212968,0.250266,0.360022,1.820315,1.004475,1,0,0
1,0.678147,-0.540678,0.340365,0.307166,1.119470,1.058917,0.745700,-1.298925,1.492134,0.514087,0.096878,0.575063,1.179122,1,0,0
2,-0.266939,0.302381,0.413461,0.755882,0.841447,-1.308210,-0.675687,-0.977004,-0.584163,2.481932,-2.095988,-1.607627,-0.853137,0,0,1
3,-1.027917,-0.791802,0.596201,-0.141550,0.285401,-0.652452,-0.285306,0.713083,-0.985464,-0.913141,2.158172,-0.544265,-1.250063,0,1,0
4,1.488220,-0.567584,-0.244401,-0.949239,1.258481,1.442775,0.965915,-0.816044,0.759324,0.570311,-0.078551,0.966827,0.702811,1,0,0


In [16]:
X_train=train.loc[:,['f1','f2','f3','f4','f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']]
y_train=train.loc[:, ['label1', 'label2', 'label3']]

In [17]:
X_test=test.loc[:,['f1','f2','f3','f4','f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']]
y_test=test.loc[:, ['label1', 'label2', 'label3']]

In [18]:
# X_train = train.iloc[:, 0:13]
# y_train = train.iloc[:, 13:17]
# y_train

In [19]:
# X_test = test.iloc[:, 0:13]
# y_test = test.iloc[:, 13:17]

In [20]:
X_train

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,1.500494,-0.558616,0.230721,-1.158640,1.884033,0.803011,1.025974,-0.655083,1.212968,0.250266,0.360022,1.820315,1.004475
1,-1.666156,-0.594491,0.925132,1.952458,-0.826691,-0.604470,-0.425443,0.310681,-0.444580,-1.064514,1.763456,0.826911,-0.592754
2,-1.960728,-1.428581,0.486557,0.456738,-0.826691,0.291200,-0.025052,0.471642,-0.270101,-0.856917,0.623166,-0.432333,-0.999206
3,0.948171,0.383099,-0.244401,0.755882,-0.687680,-1.516134,-1.356352,0.391161,-0.985464,1.954290,-1.131127,-1.313803,-0.424458
4,0.359027,-0.549647,-0.829168,-0.739839,-0.409657,0.163247,0.155123,-0.735563,-0.427132,-0.480648,0.272307,0.211281,1.703064
5,0.346753,-0.325429,1.144419,-0.799667,0.146389,1.122893,1.196140,-0.413642,0.113752,0.405964,0.491594,0.309222,1.655432
6,-0.181022,-0.881489,-0.171306,-0.440694,1.536505,-1.260228,-0.785795,-1.218445,-1.142494,-0.415774,-0.867983,-1.859476,-0.376827
7,1.684601,1.127503,-0.317497,-1.038983,0.146389,1.522746,1.136081,-0.735563,1.038490,-0.069779,0.360022,1.148718,1.004475
8,1.046362,-0.881489,-0.354045,-1.038983,-0.131634,1.090905,1.116061,-1.137965,0.445262,0.933606,0.228450,1.302626,0.940966
9,1.475946,1.531095,0.267269,-0.171465,0.771941,0.882982,0.615573,-0.494122,-0.601610,0.077269,-0.385552,0.994810,1.052106


In [21]:
X_train = np.array(X_train).astype(np.float32)
X_test  = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test  = np.array(y_test).astype(np.float32)

In [22]:
#Check to make sure split still has 4 features and 3 labels
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(151, 13) (151, 3)
(28, 13) (28, 3)


In [23]:
y_test


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [24]:
training_size = X_train.shape[1]
test_size = X_test.shape[1]
num_features = 13
num_labels = 3


num_hidden = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_set    = tf.constant(X_train)
    tf_train_labels = tf.constant(y_train)
    tf_valid_set    = tf.constant(X_test)
 
    
    print(tf_train_set)
    print(tf_train_labels)
    
    #input -> hidden
    w1 = tf.Variable(tf.truncated_normal([num_features, num_hidden]))
    b1 = tf.Variable(tf.zeros([num_hidden]))
    
    #hidden -> o/p
    w2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))


    
    z1 = tf.matmul(tf_train_set , w1 ) + b1
    a1 = tf.nn.sigmoid(z1)
    z2 = tf.matmul(a1, w2) + b2
    
    
    loss = tf.reduce_mean(tf.square(tf.subtract(z2, y_train)))
    optimizer = tf.train.GradientDescentOptimizer(.005).minimize(loss)
    
    
    ## Training prediction
    predict_train = tf.nn.sigmoid(z2)
    
    
    
    # Validation prediction
    z1_val = tf.matmul(tf_valid_set, w1) + b1
    a1_val    = tf.nn.sigmoid(z1_val)
    z2_val = tf.matmul(a1_val, w2) + b2
    predict_valid = tf.nn.sigmoid(z2_val)

Tensor("Const:0", shape=(151, 13), dtype=float32)
Tensor("Const_1:0", shape=(151, 3), dtype=float32)


In [25]:
def accuracy(predictions, labels):
    acc = (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
    return acc

num_steps = 50000
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print(loss.eval())
    for step in range(num_steps):
        _,l, predictions = session.run([optimizer, loss, predict_train])
        
        if (step % 1000 == 0):
              #print(predictions[3:6])
            print('Loss at step %d: %f' % (step, l))
            acc = accuracy( predictions, y_train[:, :])
            print('Training accuracy: %.4f%%' % acc)
            acc_valid = accuracy(predict_valid.eval(), y_test)
            print('Validation accuracy: %.4f%%' % acc_valid)
        if (acc > 99.0 and acc_valid > 99.0):
            break

Instructions for updating:
Use `tf.global_variables_initializer` instead.
2.6899476
Loss at step 0: 2.689948
Training accuracy: 5.9603%
Validation accuracy: 14.2857%
Loss at step 1000: 0.202931
Training accuracy: 79.4702%
Validation accuracy: 89.2857%
Loss at step 2000: 0.111987
Training accuracy: 90.7285%
Validation accuracy: 100.0000%
Loss at step 3000: 0.085093
Training accuracy: 96.0265%
Validation accuracy: 100.0000%
Loss at step 4000: 0.071596
Training accuracy: 96.0265%
Validation accuracy: 100.0000%
Loss at step 5000: 0.063172
Training accuracy: 96.0265%
Validation accuracy: 100.0000%
Loss at step 6000: 0.057231
Training accuracy: 96.0265%
Validation accuracy: 100.0000%
Loss at step 7000: 0.052706
Training accuracy: 96.6887%
Validation accuracy: 100.0000%
Loss at step 8000: 0.049083
Training accuracy: 96.6887%
Validation accuracy: 100.0000%
Loss at step 9000: 0.046080
Training accuracy: 96.6887%
Validation accuracy: 100.0000%
Loss at step 10000: 0.043529
Training accuracy: 97.3